<a href="https://colab.research.google.com/github/Adrita2211/ML_Project/blob/object_detection/image_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 40.8 MB/s eta 0:00:00


In [8]:
import cv2
import numpy as np
import faiss
import os
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from google.colab.patches import cv2_imshow
from google.colab import drive
import os

In [9]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load pre-trained model and remove top layers
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [6]:
# Compute embeddings using VGG16
def compute_embeddings(images):
    embeddings = []
    for img in images:
        img = cv2.resize(img, (224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        img_embedding = model.predict(img)
        embeddings.append(img_embedding.flatten())
    return np.array(embeddings)

In [7]:
def create_index(embeddings):
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings)
    return index

In [10]:
def load_images_from_folder(folder_path, target_size=(224, 224)):
    images = []
    image_paths = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            # Read image (modify based on your image types)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize if needed
                img = cv2.resize(img, target_size)
                # Convert color space if needed (BGR to RGB)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)
                image_paths.append(img_path)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
    return np.array(images), image_paths

In [11]:
folder_path = '/content/sample_data'  # Update this path
images, image_paths = load_images_from_folder(folder_path)
embeddings = compute_embeddings(images)
index = create_index(embeddings)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 716ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step


In [12]:
def search_similar_images(index, query_embedding, top_k=1):
    D, I = index.search(query_embedding, top_k)
    return I

In [13]:
print("Similar images indices:", similar_images_indices)
for idx in similar_images_indices[0]:
    print(image_paths[idx])

NameError: name 'similar_images_indices' is not defined